# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# The path to our CSV file
file = ("../output_data/clean_city_data.csv")

# Read CSV weather data into pandas
weather_data = pd.read_csv(file)
clean_data = weather_data.drop(['Unnamed: 0'],axis=1)
clean_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,bethel,41.37,-73.41,69.80,77,75,4.70,US,1596853639
1,tasiilaq,65.61,-37.64,44.60,75,21,18.34,GL,1596853639
2,te anau,-45.42,167.72,45.39,63,5,1.10,NZ,1596853423
3,east london,-33.02,27.91,54.10,69,49,9.31,ZA,1596853639
4,grand gaube,-20.01,57.66,64.40,88,40,3.36,MU,1596853639
...,...,...,...,...,...,...,...,...,...
565,aykhal,66.00,111.50,64.40,68,1,2.24,RU,1596853689
566,iowa city,41.66,-91.53,75.20,69,1,10.29,US,1596853689
567,dhidhdhoo,6.88,73.10,82.45,75,100,16.89,MV,1596853466
568,san felipe,10.34,-68.74,73.00,91,66,1.48,VE,1596853689


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key = g_key)

# Store latitude and longitude in locations
locations = clean_data[['Lat','Lng']].astype(float)

# Store Humidity in humidity
humidity = clean_data["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=4)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 100,point_radius = 1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrow_city_df = clean_data.loc[(clean_data["Wind Speed"] <= 10) & (clean_data["Cloudiness"] == 0) & (clean_data["Max Temp"] >= 60) & (clean_data["Max Temp"] <= 80)].dropna()
narrow_city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,tirschenreuth,49.88,12.33,62.01,89,0,2.48,DE,1596853641
38,kampene,-3.60,26.67,69.69,68,0,1.45,CD,1596853643
41,mazamari,-11.33,-74.53,69.33,60,0,1.92,PE,1596853643
44,fortuna,40.60,-124.16,64.00,72,0,1.01,US,1596853468
61,lokosovo,61.13,74.82,66.31,60,0,4.52,RU,1596853644


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = narrow_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the dataframe
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
21,tirschenreuth,DE,49.88,12.33,
38,kampene,CD,-3.60,26.67,
41,mazamari,PE,-11.33,-74.53,
44,fortuna,US,40.60,-124.16,
61,lokosovo,RU,61.13,74.82,


In [12]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params = params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

# Print search complete statement 
print("-------Search Complete-------")

Retrieving Results for Index 21: tirschenreuth.
Closest hotel in tirschenreuth is Hotel & Restaurant Seenario.
------------
Retrieving Results for Index 38: kampene.
Missing field/result... skipping.
------------
Retrieving Results for Index 41: mazamari.
Missing field/result... skipping.
------------
Retrieving Results for Index 44: fortuna.
Closest hotel in fortuna is Bear River Casino Resort.
------------
Retrieving Results for Index 61: lokosovo.
Missing field/result... skipping.
------------
Retrieving Results for Index 63: carutapera.
Closest hotel in carutapera is HOTEL R. SOARES.
------------
Retrieving Results for Index 81: surt.
Closest hotel in surt is City Hotel.
------------
Retrieving Results for Index 101: touros.
Closest hotel in touros is Casa Imperial Luxo.
------------
Retrieving Results for Index 133: meghri.
Closest hotel in meghri is Marishok Hotel.
------------
Retrieving Results for Index 151: okha.
Closest hotel in okha is Gostinitsa "Sakhalin-Sfera".
---------

In [14]:
# Display the hotel dataframe - *need to drop the missing fields*
hotel_df.dropna()

,City,Country,Lat,Lng,Hotel Name
21,tirschenreuth,DE,49.88,12.33,Hotel & Restaurant Seenario
38,kampene,CD,-3.60,26.67,
41,mazamari,PE,-11.33,-74.53,
44,fortuna,US,40.60,-124.16,Bear River Casino Resort
61,lokosovo,RU,61.13,74.82,
63,carutapera,BR,-1.20,-46.02,HOTEL R. SOARES
81,surt,LY,31.21,16.59,City Hotel
101,touros,BR,-5.20,-35.46,Casa Imperial Luxo
133,meghri,AM,38.90,46.24,Marishok Hotel
151,okha,RU,53.57,142.95,"Gostinitsa ""Sakhalin-Sfera"""


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))